In [59]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

[]


In [60]:
training_data = [
                 [ "Green", 3, "Apple" ],
                 ["Yellow", 3, "Apple"],
                 ["Red" ,1 , "Grape"], 
                 ["Red", 1, "Grape"] 
                ]

In [61]:
training_data

[['Green', 3, 'Apple'],
 ['Yellow', 3, 'Apple'],
 ['Red', 1, 'Grape'],
 ['Red', 1, 'Grape']]

In [62]:
header = [ "color", "diameter", "label"]

In [63]:
def unique_vals(rows, col):
    return set( [row[col] for row in rows ] )

In [64]:
unique_vals(training_data, 2)

{'Apple', 'Grape'}

In [65]:
def is_numeric(value):
    return isinstance( value, int ) or isinstance( value, float )

In [66]:
class Question:
    
    def __init__(self, column, value):
        self.column = column
        self.value = value
        
        
    def match(self, example):
        
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value
        
    def __repr__(self):
        
        condition =  "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is {} {} {}".format( header[self.column], condition, str(self.value))

In [67]:
Question( 0, "GERR" )

Is color == GERR

In [68]:
example = training_data[0]

In [69]:
example

['Green', 3, 'Apple']

In [70]:
q = Question( 0, "Green" )

In [71]:
q.match( example )

True

In [72]:
def partition(rows, question):
    
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [73]:

partition( training_data, Question(0, "Red") )

([['Red', 1, 'Grape'], ['Red', 1, 'Grape']],
 [['Green', 3, 'Apple'], ['Yellow', 3, 'Apple']])

In [74]:
Question(0, "Red").match(training_data[3])

True

In [75]:
def class_counts(rows):
    counts = {}
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [76]:
class_counts(training_data)

{'Apple': 2, 'Grape': 2}

In [77]:
def gini(rows):
    
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [78]:
gini( training_data )

0.5

In [79]:
def info_gain( left, right, current_uncertainty ):
    
    p = float( len(left) )/ ( len(left)  + len(right) )
    return current_uncertainty - p * gini(left) - ( 1 - p ) * gini( right )

In [80]:
current_uncertainty = gini( training_data )

In [81]:
current_uncertainty

0.5

In [82]:
true_rows, false_rows = partition( training_data, Question(0, "Green") )

In [83]:
info_gain( true_rows, false_rows, current_uncertainty )

0.16666666666666669

In [84]:
true_rows, false_rows = partition( training_data, Question(0, "Red") )

In [85]:
info_gain( true_rows, false_rows, current_uncertainty )

0.5

In [88]:
def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  # number of columns

    for col in range(n_features):  # for each feature

        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value

            question = Question(col, val)

            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our
            # toy dataset.
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [97]:
find_best_split(training_data)

(0.5, Is diameter >= 3)

In [90]:
class Leaf:
    
    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [91]:
class Decision_node:
    def __init__(self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [98]:
def build_tree(rows):
    from IPython.core.debugger import Pdb; Pdb().set_trace()
    gain, question = find_best_split(rows)
    
    if gain == 0:
        return Leaf(rows)
    
    true_rows, false_rows = partition(rows, question)
    
    true_branch = build_tree(true_rows)
    
    false_branch = build_tree(false_rows)
    
    
    return Decision_node(question, true_branch, false_branch)
    

In [101]:
def print_tree(node, spacing=""):
    
    if isinstance(node, Leaf):
        print(spacing + "Predict", node.predictions)
        return
    
    print( spacing + str(node.question) )
    
    print( spacing + '--> True:' )
    print_tree( node.true_branch, spacing + " ")
    
    print( spacing + "--> False: " )
    print_tree( node.false_branch, spacing + " ")
    
    

In [99]:
my_tree = build_tree(  training_data )

> <ipython-input-98-b1e8666dae0c>(3)build_tree()
      1 def build_tree(rows):
      2     from IPython.core.debugger import Pdb; Pdb().set_trace()
----> 3     gain, question = find_best_split(rows)
      4 
      5     if gain == 0:

ipdb> rows
[['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Red', 1, 'Grape'], ['Red', 1, 'Grape']]
ipdb> n
> <ipython-input-98-b1e8666dae0c>(5)build_tree()
      3     gain, question = find_best_split(rows)
      4 
----> 5     if gain == 0:
      6         return Leaf(rows)
      7 

ipdb> gain
0.5
ipdb> question
Is diameter >= 3
ipdb> n
> <ipython-input-98-b1e8666dae0c>(8)build_tree()
      6         return Leaf(rows)
      7 
----> 8     true_rows, false_rows = partition(rows, question)
      9 
     10     true_branch = build_tree(true_rows)

ipdb> n
> <ipython-input-98-b1e8666dae0c>(10)build_tree()
      8     true_rows, false_rows = partition(rows, question)
      9 
---> 10     true_branch = build_tree(true_rows)
     11 
     12     false_branch

ipdb> n
> <ipython-input-98-b1e8666dae0c>(15)build_tree()
     12     false_branch = build_tree(false_rows)
     13 
     14 
---> 15     return Decision_node(question, true_branch, false_branch)
     16 

ipdb> false_branch
ipdb> false_branch.predictions
{'Grape': 2}
ipdb> n
--Return--
<__main__.Dec...x7f08f01a2390>
> <ipython-input-98-b1e8666dae0c>(15)build_tree()
     12     false_branch = build_tree(false_rows)
     13 
     14 
---> 15     return Decision_node(question, true_branch, false_branch)
     16 

ipdb> n
--Return--
None
> <ipython-input-99-0f588aa76e20>(1)<module>()
----> 1 my_tree = build_tree(  training_data )

ipdb> c


In [102]:
print_tree( my_tree )

Is diameter >= 3
--> True:
 Predict {'Apple': 2}
--> False: 
 Predict {'Grape': 2}


In [96]:
training_data

[['Green', 3, 'Apple'],
 ['Yellow', 3, 'Apple'],
 ['Red', 1, 'Grape'],
 ['Red', 1, 'Grape']]